In [63]:
import gym
import numpy as np
import time
from tqdm import tqdm
from IPython.display import clear_output
from gym.envs.toy_text.taxi import MAP as taxi_map
import imageio

In [2]:
action_names = [
    'South (↓)',
    'North (↑)',
    'East (→)',
    'West (←)',
    'Pickup',
    'Drop off',
]

In [61]:
def play(env_name, policy=None, sleep_time=0.1, env_seed=None, recording = False):
    env = gym.make(env_name) 
    #这里原来把环境make放在外面的，但是一关环境还得重新打开，就放里面了，让每次运行都重新打开这个环境，窗口开关看着舒服
    
    if env_seed is not None:
        env.seed(env_seed)
    
    #没有策略的话就直接随便选
    if policy is None:
        policy=lambda s: env.action_space.sample()
        
    states = []
    rewards = []
    actions = []
    #gif用来将结果保存
    gif= []
    state = env.reset()
    states.append(state)
    max_steps = env.spec.max_episode_steps
    total_reward = 0
    current_step = 0
    is_done = False
    
    while not is_done:
        #将env渲染的结果以array的格式保存，并存储到gif列表中
        picture = env.render(mode = 'rgb_array')
        gif.append(picture)
        
        action = policy(state)
        state, reward, is_done, info = env.step(action)
        
        states.append(state)
        rewards.append(reward)
        actions.append(action)
        total_reward += reward
        current_step +=1
        
        clear_output(wait=True)
        print('Step: {:03d}/{}, Reward: {}\n'.format(
            current_step,
            max_steps,
            total_reward,
        ))
        
    picture = env.render(mode = 'rgb_array')
    gif.append(picture)
    
    #如果需要记录，则生成相应gif
    if recording == True:
         imageio.mimsave("taxi.gif", gif, fps=5)
            
    if current_step < max_steps:
        print('\nResult: Done with {} steps and total reward is {}.'.format(
            current_step,
            total_reward,
        ))
    else:
        print('\nResult:Unsolved')
                        
    env.close()    
    return states, rewards, actions

In [5]:
states, rewards, actions = play('Taxi-v3', sleep_time=0.01, env_seed=1)

Step: 200/200, Reward: -713


Result:Unsolved


In [6]:
def Q_learning(env_name, learning_rate = 0.1, discount_factor = 0.99, epsilon = 0.1, training_episodes = 100000):#脑子抽了，在这里加了个seed，后面错了
    env = gym.make(env_name)
    q_table_shape = [env.observation_space.n, env.action_space.n]
    q_table = np.zeros(q_table_shape)
    rng = np.random.default_rng()
    for i in tqdm(range(training_episodes)):
        state = env.reset()
        done = False
        while not done:
            #epsilon贪心策略，不过直接0.1也不是特别的好，后续改进让ε随时间越来越小
            if rng.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state])
                
            next_state, reward, done, info = env.step(action)
            #更新
            q_table[state, action] = (1 - learning_rate) * q_table[state][action] + learning_rate * (reward + discount_factor * np.max(q_table[next_state]))
            
            state = next_state
    return q_table

In [7]:
q_table = Q_learning('Taxi-v3')#训练

100%|██████████| 100000/100000 [00:58<00:00, 1717.69it/s]


In [62]:
#检验结果，并保存为gif
play(
    'Taxi-v3',
    policy=lambda s: np.argmax(q_table[s]),
    sleep_time=0.5,
    recording = True
)

Step: 011/200, Reward: 10


Result: Done with 11 steps and total reward is 10.


([392, 372, 472, 476, 376, 276, 256, 236, 216, 116, 16, 0],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 20],
 [3, 0, 4, 1, 1, 3, 3, 3, 1, 1, 5])

env = gym.make('Taxi-v3')  
state = env.reset()  
scene = env.render(mode = 'rgb_array')  
print(scene)  
看了看这啥格式的

def checkPic(Carray):  
    rows = Carray.shape[0]  
    cols = Carray.shape[1]  
    image = Image.new("RGB",(rows,cols))  
    for i in range(rows):  
        for j in range(cols):  
            image.putpixel((i,j),(Carray[i,j,0],Carray[i,j,1],Carray[i,j,2]))  
    image = image.rotate(270, expand=True)  
    #image.show()  
一开始用这个把array转为图片的